In [179]:
# Note: Wireshark previous segment not captured
# might be 'packet loss' or Wireshark not capture 'fast enough'

import csv

def read_from_csv(csv_filename):

    frame_list = []
    seq_list = []
    
    with open(csv_filename, newline='') as csvfile:

        rows = csv.DictReader(csvfile)

        for row in rows:
            frame_list.append(int(row['frame.number']))

            #this is UDP packet
            if(row['tcp.seq'] == ''):
                seq_list.append(-1)
            else:
                seq_list.append(int(row['tcp.seq']))
    
    return frame_list, seq_list

In [180]:
# Two pointers check packet loss
# Fast pointer check send list
# Slow pointer check receive list

# Divide UDP and (TCP + TLS) packet
def udp_tcptls_packet_count(send_seq_list):
    
    udp_count = 0
    tcptls_count = 0
    
    for seq in send_seq_list:
        if(seq == -1): udp_count = udp_count + 1
        else: tcptls_count = tcptls_count + 1
    
    return udp_count, tcptls_count


def calculate_pkt_loss(send_seq_list, send_frame_list, receive_seq_list, receive_frame_list):
    
    udp_count, tcptls_count = udp_tcptls_packet_count(send_seq_list)
    print("Number of UDP packet: " + str(udp_count))
    print("Number of TCP + TLS packet: " + str(tcptls_count))
    
    tcptls_packetloss_frameNum_seqNum = []
    udp_packetloss_frameNum_seqNum = []
    s_ptr = 0
    
    pair_frameNum_list = []
    
    # Detect too many consecutive packet loss
    # maybe something went wrong
    consec_loss_packet_count = 0
    
    for f_ptr in range(0, len(send_seq_list)):
 
        if(send_seq_list[f_ptr] != receive_seq_list[s_ptr]):
            
            # Deal with 'Spurious Retransmission'
            if(receive_seq_list[s_ptr] == receive_seq_list[s_ptr-1]):
                print("\nSuprious Retransmission? Sender Frame #: " + str(send_frame_list[f_ptr]) + ", Seq: " + str(send_seq_list[f_ptr]))
                print("Suprious Retransmission? Receive Frame #: " + str(receive_frame_list[s_ptr]) + ", Seq: " + str(receive_seq_list[s_ptr]))
                s_ptr = s_ptr + 1
                
                if(send_seq_list[f_ptr] == receive_seq_list[s_ptr]):
                    pair_frameNum_list.append([send_frame_list[f_ptr], receive_frame_list[s_ptr]])
                    
                    s_ptr = s_ptr + 1
                    consec_loss_packet_count = 0
                    
                    # Finish
                    if(s_ptr >= len(receive_seq_list)):
                        break
                        
                    continue
                
            if(send_seq_list[f_ptr] == -1):
                udp_packetloss_frameNum_seqNum.append([send_frame_list[f_ptr], send_seq_list[f_ptr],
                                                          receive_frame_list[s_ptr-1], receive_seq_list[s_ptr-1]])
            
            else:
                tcptls_packetloss_frameNum_seqNum.append([send_frame_list[f_ptr], send_seq_list[f_ptr],
                                                          receive_frame_list[s_ptr-1], receive_seq_list[s_ptr-1]])
            
            consec_loss_packet_count = consec_loss_packet_count + 1
            
            if(consec_loss_packet_count > 10):
                print("Error! Too many consecutive packet loss!\n")
                print("Now is checking sender frame num: " + str(send_frame_list[f_ptr]) + ", seq: " + str(send_seq_list[f_ptr]))
                print("Now is checking receiver frame num: " + str(receive_frame_list[s_ptr]) + ", seq: " + str(receive_seq_list[s_ptr]) + "\n")
                break
            
        else:
            pair_frameNum_list.append([send_frame_list[f_ptr], receive_frame_list[s_ptr]])
            
            s_ptr = s_ptr + 1
            consec_loss_packet_count = 0
            
            # Finish
            if(s_ptr >= len(receive_seq_list)):
                break

    tcptls_loss_rate = len(tcptls_packetloss_frameNum_seqNum)/tcptls_count
    udp_loss_rate = len(udp_packetloss_frameNum_seqNum)/udp_count
    
    return [tcptls_packetloss_frameNum_seqNum, tcptls_loss_rate, udp_packetloss_frameNum_seqNum, udp_loss_rate, pair_frameNum_list]

In [208]:
def start_calc(filepath):
    
    evcc_send_frame_list, evcc_send_seq_list = read_from_csv(filepath + 'evcc_send.csv')
    secc_receive_frame_list, secc_receive_seq_list = read_from_csv(filepath + 'secc_receive.csv')
    
    print("Calculating EVCC send packet loss: ")
    evcc_send_packet_lost_list = calculate_pkt_loss(evcc_send_seq_list, evcc_send_frame_list, secc_receive_seq_list, secc_receive_frame_list)
    print("UDP lost: " + str(len(evcc_send_packet_lost_list[2])))
    print("UDP lost rate: " + str(evcc_send_packet_lost_list[3]))
    print("TCP+TLS lost: " + str(len(evcc_send_packet_lost_list[0])))
    print("TCP+TLS lost rate: " + str(evcc_send_packet_lost_list[1]))
    
    evcc_receive_frame_list, evcc_receive_seq_list = read_from_csv(filepath + 'evcc_receive.csv')
    secc_send_frame_list, secc_send_seq_list = read_from_csv(filepath + 'secc_send.csv')
    
    print("\nCalculating SECC send packet loss: ")
    secc_send_packet_lost_list = calculate_pkt_loss(secc_send_seq_list, secc_send_frame_list, evcc_receive_seq_list, evcc_receive_frame_list)
    print("UDP lost: " + str(len(secc_send_packet_lost_list[2])))
    print("UDP lost rate: " + str(secc_send_packet_lost_list[3]))
    print("TCP+TLS lost: " + str(len(secc_send_packet_lost_list[0])))
    print("TCP+TLS lost rate: " + str(secc_send_packet_lost_list[1]))
    
    # UDP + (TCP & TLS) total
    print("\nCalculate total packet loss rate: ")
    total_packet_loss = len(evcc_send_packet_lost_list[2]) + len(evcc_send_packet_lost_list[0]) + len(secc_send_packet_lost_list[2]) + len(secc_send_packet_lost_list[0])
    total_packet_loss_rate = total_packet_loss/(len(evcc_send_frame_list) + len(secc_send_frame_list))
    print("Total packet loss: " + str(total_packet_loss))
    print("Total packet loss rate: " + str(total_packet_loss_rate))
    
    # TCP & TLS total
    print("\nCalculate total TCP + TLS packet loss rate: ")
    total_tcptls_packet_loss = len(evcc_send_packet_lost_list[0]) + len(secc_send_packet_lost_list[0])
    total_tcptls_packet_loss_rate = total_packet_loss/(udp_tcptls_packet_count(evcc_send_seq_list)[1] + udp_tcptls_packet_count(secc_receive_seq_list)[1])
    print("Total packet loss: " + str(total_tcptls_packet_loss))
    print("Total packet loss rate: " + str(total_tcptls_packet_loss_rate))
    
    return [evcc_send_packet_lost_list, secc_send_packet_lost_list]

In [209]:
packetlost_list_0420_interf4 = start_calc('0420/interf_4/')

Calculating EVCC send packet loss: 
Number of UDP packet: 52
Number of TCP + TLS packet: 1221
UDP lost: 25
UDP lost rate: 0.4807692307692308
TCP+TLS lost: 63
TCP+TLS lost rate: 0.051597051597051594

Calculating SECC send packet loss: 
Number of UDP packet: 27
Number of TCP + TLS packet: 1222
UDP lost: 1
UDP lost rate: 0.037037037037037035
TCP+TLS lost: 11
TCP+TLS lost rate: 0.00900163666121113

Calculate total packet loss rate: 
Total packet loss: 100
Total packet loss rate: 0.03965107057890563

Calculate total TCP + TLS packet loss rate: 
Total packet loss: 74
Total packet loss rate: 0.042034468263976464


In [210]:
packetlost_list_0420_interf2 = start_calc('0420/interf_2/')

Calculating EVCC send packet loss: 
Number of UDP packet: 42
Number of TCP + TLS packet: 1137
UDP lost: 7
UDP lost rate: 0.16666666666666666
TCP+TLS lost: 36
TCP+TLS lost rate: 0.0316622691292876

Calculating SECC send packet loss: 
Number of UDP packet: 35
Number of TCP + TLS packet: 1161
UDP lost: 1
UDP lost rate: 0.02857142857142857
TCP+TLS lost: 15
TCP+TLS lost rate: 0.012919896640826873

Calculate total packet loss rate: 
Total packet loss: 59
Total packet loss rate: 0.024842105263157895

Calculate total TCP + TLS packet loss rate: 
Total packet loss: 51
Total packet loss rate: 0.026362823949955316


In [211]:
packetlost_list_0420_interf0 = start_calc('0420/interf_0/')

Calculating EVCC send packet loss: 
Number of UDP packet: 42
Number of TCP + TLS packet: 1249

Suprious Retransmission? Sender Frame #: 3810, Seq: 3950
Suprious Retransmission? Receive Frame #: 2533, Seq: 3881
UDP lost: 14
UDP lost rate: 0.3333333333333333
TCP+TLS lost: 50
TCP+TLS lost rate: 0.040032025620496396

Calculating SECC send packet loss: 
Number of UDP packet: 28
Number of TCP + TLS packet: 1260
UDP lost: 0
UDP lost rate: 0.0
TCP+TLS lost: 15
TCP+TLS lost rate: 0.011904761904761904

Calculate total packet loss rate: 
Total packet loss: 79
Total packet loss rate: 0.030632027917797598

Calculate total TCP + TLS packet loss rate: 
Total packet loss: 65
Total packet loss rate: 0.03225806451612903


In [ ]:
# TODO:
# whether to process 'Previous segment not captured'?

In [212]:
packetlost_list_0612_no_interf = start_calc('0612/no_interf/')

Calculating EVCC send packet loss: 
Number of UDP packet: 20
Number of TCP + TLS packet: 931
UDP lost: 0
UDP lost rate: 0.0
TCP+TLS lost: 0
TCP+TLS lost rate: 0.0

Calculating SECC send packet loss: 
Number of UDP packet: 20
Number of TCP + TLS packet: 997
UDP lost: 0
UDP lost rate: 0.0
TCP+TLS lost: 11
TCP+TLS lost rate: 0.011033099297893681

Calculate total packet loss rate: 
Total packet loss: 11
Total packet loss rate: 0.005589430894308943

Calculate total TCP + TLS packet loss rate: 
Total packet loss: 11
Total packet loss rate: 0.005907626208378088


In [213]:
packetlost_list_0612_max_interf = start_calc('0612/max_interf/')

Calculating EVCC send packet loss: 
Number of UDP packet: 26
Number of TCP + TLS packet: 946

Suprious Retransmission? Sender Frame #: 166, Seq: 336
Suprious Retransmission? Receive Frame #: 212, Seq: 245
UDP lost: 1
UDP lost rate: 0.038461538461538464
TCP+TLS lost: 3
TCP+TLS lost rate: 0.003171247357293869

Calculating SECC send packet loss: 
Number of UDP packet: 25
Number of TCP + TLS packet: 1021
UDP lost: 0
UDP lost rate: 0.0
TCP+TLS lost: 26
TCP+TLS lost rate: 0.025465230166503428

Calculate total packet loss rate: 
Total packet loss: 30
Total packet loss rate: 0.014866204162537165

Calculate total TCP + TLS packet loss rate: 
Total packet loss: 29
Total packet loss rate: 0.015873015873015872


In [214]:
packetlost_list_0612_max_interf_VI = start_calc('0612/max_interf_VI/')

Calculating EVCC send packet loss: 
Number of UDP packet: 23
Number of TCP + TLS packet: 925
UDP lost: 0
UDP lost rate: 0.0
TCP+TLS lost: 3
TCP+TLS lost rate: 0.003243243243243243

Calculating SECC send packet loss: 
Number of UDP packet: 23
Number of TCP + TLS packet: 1003
UDP lost: 0
UDP lost rate: 0.0
TCP+TLS lost: 25
TCP+TLS lost rate: 0.024925224327018942

Calculate total packet loss rate: 
Total packet loss: 28
Total packet loss rate: 0.014184397163120567

Calculate total TCP + TLS packet loss rate: 
Total packet loss: 28
Total packet loss rate: 0.015159718462371413


In [215]:
# There are out-of-order packets in evcc receive
# I have changed the ordering in the csv file
packetlost_list_0612_max_interf_VO = start_calc('0612/max_interf_VO/')

Calculating EVCC send packet loss: 
Number of UDP packet: 22
Number of TCP + TLS packet: 937
UDP lost: 0
UDP lost rate: 0.0
TCP+TLS lost: 2
TCP+TLS lost rate: 0.0021344717182497333

Calculating SECC send packet loss: 
Number of UDP packet: 22
Number of TCP + TLS packet: 1001
UDP lost: 0
UDP lost rate: 0.0
TCP+TLS lost: 20
TCP+TLS lost rate: 0.01998001998001998

Calculate total packet loss rate: 
Total packet loss: 22
Total packet loss rate: 0.011099899091826439

Calculate total TCP + TLS packet loss rate: 
Total packet loss: 22
Total packet loss rate: 0.011752136752136752
